In [1]:
from datasets import load_dataset

ds = load_dataset('openpecha/stt-training-data')
ds

DatasetDict({
    train: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len'],
        num_rows: 1362015
    })
    validation: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len'],
        num_rows: 20742
    })
})

In [2]:
tt = ds.filter(lambda x: x["dept"] == "STT_TT" and x['uni'].strip() != '')

In [3]:
tt

DatasetDict({
    train: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len'],
        num_rows: 446495
    })
    validation: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len'],
        num_rows: 6800
    })
})

In [4]:
tt['train']['uni']

['དཔེར་ན་ང་ཚོ་སྤྱི་ཡོངས་ནས་འཛམ་བུ་གླིང་ལ་བསམ་བློ་བཏང་ཡོང་དུས་དེ་རིང་ཁ་སང་ཛམ་བུ་གླིང་གི་ ',
 'གནས་ཡ་གི་གནས་སྐབས་ལ།  ',
 'ད་དེ་ནས། ཨ་མ་ལ་ཀ་ཤ་པར་ན། གསེར་གྱི་ཀ་ཤ་པར་ན་སུམ་ཅུ་སུམ་ཅུ་སོ་གཉིས་ཕུལ་རེད་བ། དེའི་ལས་ཀྱི་འབྲས་བུ་དེ།  ',
 'སྤྲུལ་པའི་སྐུ་ཡང་དབྱིངས་སུ་ཐིམ། མདོར་ན།  ',
 'ཨ། དེ་འདྲའི་ཟེར་གི་རེད་དེ། གང་བྱས་སྐྱེ་རུང་མི་རུང་ཟེར་ཡ་དེ། སྐྱེ་བའི་ཁྱབ་བ་མ། ངེས་མ་ངེས་ལ་མ་རེད་ཟེ།  ',
 'སྐད་བསྒྱུར་སྐད་བསྒྱུར་རེ་ད་གང་ཟེ་སྐད་བསྒྱུར་རེ་ཁ་ཐུག་ལ་ངས་ཧབ་སྦྲིད་བརྒྱག་པ་རེད་ ',
 'ལམ་སང་འགོ་བཙུགས་ཡོང་བསྡད་ཀི་འདུག་བ། དེ་འདྲའི་མ་བྱེད་གསུངས་པ་རེད་ད་དེ།  ',
 'ལས་འབྲས་ཁྱད་དུ་བསད་པའི་རྒྱུ་མཐུན་གི་འབྲས་བུ་དེ་ཆགས་བསྡད་ཡོད་རེད། ད་དེའི་གཞུག་གུ་དེ་ལ།  ',
 'ཨེ་ནས། དེ་འདྲའི་བདེ་བ་ཆེན་པོའི་ཡེ་ཤེས་ལ་སྙིང་རྗེ་ཟེ་ནས། མིང་བཏགས་པ་ཙམ་རྒྱུ་མཚན་དུ་བྱས་ནས།  ',
 'ཨ་ཁྱེན་ཚར་ ',
 'བྱས་ཙང་ག་ང་རང་ཚོའི་ལོ་རྒྱུས་ཁ་ཤས་ཨེ་ནས་ཁོ།  ',
 'དེ་འདྲའི། ཅིག ནུས་པ་སྤོབས་ལོད་དེ་ཡེ་མེད་པ་བྱས།  ',
 'ས་དགེ་བཀའ་རྙིང་རིས་སུ་མ་ཆད་པའི་ ',
 'འདོད་པ་རྣམས་ནི་ཕུང་བ་བསྐྱེད་པ་སྟེ། རྒྱལ་བའི་དབང་པོས་ ',
 'ཤིང་རྟ་ཆེན་ ',
 'རྟོག་གེ་འབར་བ་ལས། གང་ཚེ་དགེ་བ་ཡང་།  ',
 'འད

In [5]:
from transformers import pipeline

pipe = pipeline('translation', 'billingsmoore/mlotsawa-ground-base')

/home/j/Desktop/MLotsawa/.venv/lib/python3.12/site-packages/transformers/pipelines/__init__.py:1154: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_bo_to_en"
  warnings.warn(
Device set to use cuda:0


## Generate Machine Translations

In [7]:
def get_mt(example):

    input = example['uni']

    output = pipe(input)

    translations = [elt['translation_text'] for elt in output]

    example['translation'] = translations
    
    return example

w_mt = tt['train'].select(range(10_000)).map(get_mt, batched=True, batch_size=8)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
w_mt.save_to_disk('stt-w-mt-poc-ds')

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]